# Probability of survival of Titanic's passengers using Decision Trees and Random Forests

Used Techniques: Supervised learning, Decision Trees and Random Forests. 

In this exercise, we are going to use a kaggle dataset coming from this URL: https://www.kaggle.com/code/faressayah/decision-trees-random-forest-for-beginners/input

In [2]:
import pandas as pd
import numpy as np

In [5]:
#Since we downloaded the data into a csv, let's convert it into a pandas dataframe.
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [15]:
#Now that our data is legible our first step is knowing both of our dataframes. 
#Magnitude of or dataframe
print(train_df.shape)
print(test_df.shape)

#Information per column
print("\n Train dataset information")
print(train_df.info())
print("\n Test dataset information")
print(test_df.info())

(891, 12)
(418, 11)

 Train dataset information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None

 Test dataset information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------      

By seeing both lists, there are several points to address: 
1. The training and testing sets the columns "Age" and "Cabin" have some null data. It's important to fix it in order to avoid any model underfitting. 
2. The columns "Name", "Sex", "Tickets", "Cabin" and "Embarked" are an object.This must be corrected as the models do not accept object type data.
3. The column "Age" is a float type data. Which by default this doesn't make any sense. 

Now that we know more about our data, let's dive more into the samples to correct each case.

In [ ]:
#Statistical description quick view
print(train_df.describe())